<a href="https://colab.research.google.com/github/teshi24/aiso/blob/main/SumFrameSudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sum Frame Sudoku Puzzle

Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.
Numbers in the outside frame equal the sum of the first three numbers in the corresponding row or column
in the given direction.

Find more examples [here](http://frame-sudoku.blogspot.com)

In [10]:
!pip install ortools

In [12]:
from ortools.sat.python import cp_model
from itertools import product

Sudoku puzzle from slides

In [22]:
top = [15, 18, 12, 11, 21, 13, 15, 17, 13]
right = [22, 8, 15, 22, 12, 11, 15, 13, 17]
bottom = [15, 9, 21, 10, 16, 19, 13, 15, 17]
left = [8, 15, 22, 11, 13, 21, 18, 19, 8]

And another example (commented out)

In [4]:
# top = [21, 12, 12, 13, 14, 18, 10, 19, 16]
# right = [20, 15, 10, 22, 8, 15, 17, 15, 13]
# bottom = [17, 9, 19, 18, 13, 14, 23, 15, 7]
# left = [12, 12, 21, 14, 14, 17, 14, 9, 22]

Create model

In [14]:
model = cp_model.CpModel()

In [48]:
# Type your model here ...
board =  [[model.NewIntVar(1, 9, f"({i},{j})") for j in range(9)] for i in range(9)]

# basic sudocu
for i in range(9):
  model.AddAllDifferent([board[i][j] for j in range(9)]) # Rows
  model.AddAllDifferent([board[j][i] for j in range(9)]) # Columns
for i in range(3):
  for j in range(3):
    model.AddAllDifferent([board[i * 3 + di][j * 3 + dj] for di in range(3) for dj in range(3)])
board_size = 9

# sum stuff
# for i in range(9):
#   if i < 3 or i >= 6:
#         model.Add(sum(board[i][j] for j in range(3)) == top[i])
#         model.Add(sum(board[j][i] for j in range(3)) == left[i])
#         model.Add(sum(board[i][j] for j in range(9-3, 9)) == bottom[i])
#         model.Add(sum(board[j][i] for j in range(9-3, 9)) == right[i])
# for i in range(9):
#         model.Add(sum(board[i][j] for j in range(3)) == left[i])
#         model.Add(sum(board[i][j] for j in range(9-3, 9)) == right[i])
#         model.Add(sum(board[j][i] for j in range(3)) == top[i])
#         model.Add(sum(board[j][i] for j in range(9-3, 9)) == bottom[i])

# for i in range(board_size):
#         if i < 3 or i >= 6:  # Apply the sum constraints only to the outer parts of the board
#             model.Add(sum(board[i][j] for j in range(3)) == left[i])
#             model.Add(sum(board[i][j] for j in range(board_size-3, board_size)) == right[i])
#             model.Add(sum(board[j][i] for j in range(3)) == top[i])
#             model.Add(sum(board[j][i] for j in range(board_size-3, board_size)) == bottom[i])

# for i in range(3):
#   for j in range(9):
#     # print(f'board item {i}, {j}; top[{j}] = {top[j]}; bottom[{j}] = {bottom[j]}')
#     if i == 0:
#       model.Add(sum(board[i * 3 + di][j] for di in range(3)) == top[j])
#     # elif i == 1:
#       # continue
#     elif i == 2:
#       model.Add(sum(board[i * 3 + di][j] for di in range(3)) == bottom[j])

# for i in range(9):
#   for j in range(3):
#     print(f'board item {i}, {j}; right[{i}] = {right[i]}; left[{i}] = {left[i]}')
#     if j == 0:
#       # print(f'left {i}: {left[i]}')
#       # for dj in range(3):
#       #   print(f'board[{i}][{j * 3 + dj}]')
#       model.Add(sum(board[i][j * 3 + dj] for dj in range(3)) == left[i])
#     # elif j == 1:
#       # continue
#     elif j == 2:
#       # print(f'right {i}: {right[i]}')
#       # for dj in range(3):
#       #   print(f'board[{i}][{j * 3 + dj}]')
#       model.Add(sum(board[i][j * 3 + dj] for dj in range(3)) == right[i])



Callback for solution printing (adapt if you do not use an n*n board)

In [38]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        print(variables)

    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{self.Value(self.__variables[i][j])}] ", end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [49]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, SolutionPrinter(board))
print(status)

[[(0,0)(1..9), (0,1)(1..9), (0,2)(1..9), (0,3)(1..9), (0,4)(1..9), (0,5)(1..9), (0,6)(1..9), (0,7)(1..9), (0,8)(1..9)], [(1,0)(1..9), (1,1)(1..9), (1,2)(1..9), (1,3)(1..9), (1,4)(1..9), (1,5)(1..9), (1,6)(1..9), (1,7)(1..9), (1,8)(1..9)], [(2,0)(1..9), (2,1)(1..9), (2,2)(1..9), (2,3)(1..9), (2,4)(1..9), (2,5)(1..9), (2,6)(1..9), (2,7)(1..9), (2,8)(1..9)], [(3,0)(1..9), (3,1)(1..9), (3,2)(1..9), (3,3)(1..9), (3,4)(1..9), (3,5)(1..9), (3,6)(1..9), (3,7)(1..9), (3,8)(1..9)], [(4,0)(1..9), (4,1)(1..9), (4,2)(1..9), (4,3)(1..9), (4,4)(1..9), (4,5)(1..9), (4,6)(1..9), (4,7)(1..9), (4,8)(1..9)], [(5,0)(1..9), (5,1)(1..9), (5,2)(1..9), (5,3)(1..9), (5,4)(1..9), (5,5)(1..9), (5,6)(1..9), (5,7)(1..9), (5,8)(1..9)], [(6,0)(1..9), (6,1)(1..9), (6,2)(1..9), (6,3)(1..9), (6,4)(1..9), (6,5)(1..9), (6,6)(1..9), (6,7)(1..9), (6,8)(1..9)], [(7,0)(1..9), (7,1)(1..9), (7,2)(1..9), (7,3)(1..9), (7,4)(1..9), (7,5)(1..9), (7,6)(1..9), (7,7)(1..9), (7,8)(1..9)], [(8,0)(1..9), (8,1)(1..9), (8,2)(1..9), (8,3)(1

In [18]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")

Runtime:   0.003072725ms
Booleans:  0
Failures:  0
Branches:  0
